In [ ]:
import nltk
from nltk.corpus import words

nltk.download('words')
word_dictionary = list(set(words.words()))


for alphabet in "bcdefghjklmnopqrstuvwxyz":
	word_dictionary.remove(alphabet)

def split_hashtag_to_words_all_possibilities(hashtag):
	all_possibilities = []
	
	split_posibility = [hashtag[:i] in word_dictionary for i in reversed(range(len(hashtag)+1))]
	possible_split_positions = [i for i, x in enumerate(split_posibility) if x == True]
	
	for split_pos in possible_split_positions:
		split_words = []
		word_1, word_2 = hashtag[:len(hashtag)-split_pos], hashtag[len(hashtag)-split_pos:]
		
		if word_2 in word_dictionary:
			split_words.append(word_1)
			split_words.append(word_2)
			all_possibilities.append(split_words)

			another_round = split_hashtag_to_words_all_possibilities(word_2)
				
			if len(another_round) > 0:
				all_possibilities = all_possibilities + [[a1] + a2 for a1, a2, in zip([word_1]*len(another_round), another_round)]
		else:
			another_round = split_hashtag_to_words_all_possibilities(word_2)
			
			if len(another_round) > 0:
				all_possibilities = all_possibilities + [[a1] + a2 for a1, a2, in zip([word_1]*len(another_round), another_round)]
	
	return all_possibilities

[nltk_data] Downloading package words to /home/srijan2022/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_without_stopwords(df , COL="tweet"):
    newdf= df.copy()

    for ind in df.index:

        a = re.sub(r'\B@[\S]+\b',' ', df[COL][ind])            # remove usernames  
        b = re.sub(r'(\bhttp:\/\/\S+\.\S+\b)|(\bwww\.\S+\.\S+\b)|(\bhttps:\/\/\S+\.\S+\b)',' ' , a)  # remove url
        c = re.sub(r'&\S+;',' ', b)        # godlike optimsitaion &#number; replaced with space
        d = re.sub(r'\.+','.', c)                   # replace multiple "."
        e = re.sub(r'!+','!', d)                    # replace multiple "!"
        f = re.sub(r'\?+','?',e)                    # replace multiple "?"
        g = re.sub(r'\"+',' ',f)                    # replace multiple " *"* "
        h = re.sub(r':',' ',g)                      # replace multiple ":"
        i = re.sub(r'\bRT\b',' ',h)                 # remove retweet keyword  "RT"

        # can remove all special characters and numbers except ' . - 

        j = re.sub(r'\s+',' ',i)                    # remove multiple whitespaces

        temp = j.lower().split()
        temp2=[]

        #print(temp)

        for i in range(len(temp)) :
            if(temp[i][0]=='#'):
                if temp[i][1:] in word_dictionary :                  # made separate case to hadle a bug #repsect not being detecetd 
                    temp2.append(temp[i][1:]) 
                else :
                        if split_hashtag_to_words_all_possibilities(temp[i][1:]) != [] :
                            temp2.extend(split_hashtag_to_words_all_possibilities(temp[i][1:])[0])
            else :
                    temp2.append(temp[i])     

        # print(temp2)       

        filtered_sentence=[]
        for w in temp2:
            if w not in stop_words:
                filtered_sentence.append(w)
        


        sentence = ' '.join([str(elem) for elem in filtered_sentence])


        newdf[COL][ind] = newdf[COL][ind].replace(newdf[COL][ind],sentence)
    return newdf

# preprocess(test_df)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/srijan2022/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_with_stopwords(df , COL="tweet"):
    newdf= df.copy()

    for ind in df.index:

        a = re.sub(r'\B@[\S]+\b',' ', df[COL][ind])            # remove usernames  
        b = re.sub(r'(\bhttp:\/\/\S+\.\S+\b)|(\bwww\.\S+\.\S+\b)|(\bhttps:\/\/\S+\.\S+\b)',' ' , a)  # remove url
        c = re.sub(r'&\S+;',' ', b)        # godlike optimsitaion &#number; replaced with space
        d = re.sub(r'\.+','.', c)                   # replace multiple "."
        e = re.sub(r'!+','!', d)                    # replace multiple "!"
        f = re.sub(r'\?+','?',e)                    # replace multiple "?"
        g = re.sub(r'\"+',' ',f)                    # replace multiple " *"* "
        h = re.sub(r':',' ',g)                      # replace multiple ":"
        i = re.sub(r'\bRT\b',' ',h)                 # remove retweet keyword  "RT"

        # can remove all special characters and numbers except ' . - 

        j = re.sub(r'\s+',' ',i)                    # remove multiple whitespaces

        temp = j.lower().split()
        temp2=[]

        #print(temp)

        for i in range(len(temp)) :
            if(temp[i][0]=='#'):
                if temp[i][1:] in word_dictionary :                  # made separate case to hadle a bug #repsect not being detecetd 
                    temp2.append(temp[i][1:]) 
                else :
                        if split_hashtag_to_words_all_possibilities(temp[i][1:]) != [] :
                            temp2.extend(split_hashtag_to_words_all_possibilities(temp[i][1:])[0])
            else :
                    temp2.append(temp[i])     

        # print(temp2)       

        # filtered_sentence=[]
        # for w in temp2:
        #     if w not in stop_words:
        #         filtered_sentence.append(w)
        


        sentence = ' '.join([str(elem) for elem in temp2])


        newdf[COL][ind] = newdf[COL][ind].replace(newdf[COL][ind],sentence)
    return newdf

# preprocess(test_df)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/srijan2022/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
